# 2018 Mw7.5 Palu, Sulawesi earthquake

## Mesh

In [ ]:
import vtk
import pyvista as pv

reader = vtk.vtkXdmfReader()
reader.SetFileName('Sulawesi_65dip_straightBay_ShortNorth_micro.xdmf')
reader.Update()
mesh = pv.wrap(reader.GetOutput())
elevation = mesh.elevation(low_point=(0, 0, -800), high_point=(0, 0, 2970))
pv.plot(elevation, cmap='terrain', background='grey', show_edges=False, jupyter_backend='static')

## Run the code

In the following we start SeisSol using the provided parameters file.
You should adjust the OMP_NUM_THREADS variable to match the number of cores on your PC.

In [ ]:
!OMP_NUM_THREADS=4 SeisSol_Release_dhsw_4_elastic parameters.par

## Visualization

We now visualize the fault output generated by SeisSol.
Check out the [documentation](https://seissol.readthedocs.io/en/latest/fault-output.html#outputmask) for an explanation of the variable names.

In [ ]:
import vtk
import pyvista as pv
from ipywidgets import interact

reader = vtk.vtkXdmfReader()
reader.SetFileName('output/Sulawesi-fault.xdmf')
reader.Update()
cd = reader.GetOutput().GetCellData()
variables = [cd.GetArrayName(i) for i in range(cd.GetNumberOfArrays())]

@interact(t=(0.0, 30.0, 0.5), var=variables)
def plot(t=0.0, var='SRs'):
    reader.UpdateTimeStep(t)
    mesh = pv.wrap(reader.GetOutput())
    plotter = pv.Plotter(notebook=True)
    plotter.set_background('grey')
    plotter.add_mesh(mesh, cmap='Blues', scalars=var)
    plotter.show(jupyter_backend='static')

In the following we visualize the free surface output.
Fields *u, v, w* are the particle velocities and *U, V, W* are the displacements in *x, y, z* direction, respectively.

In [ ]:
reader = vtk.vtkXdmfReader()
reader.SetFileName('output/Sulawesi-surface.xdmf')
reader.Update()
cd = reader.GetOutput().GetCellData()
variables = [cd.GetArrayName(i) for i in range(cd.GetNumberOfArrays())]

@interact(t=(0.0, 30.0, 0.25), var=variables)
def plot(t=0.0, var='W'):
    reader.UpdateTimeStep(t)
    mesh = pv.wrap(reader.GetOutput())
    plotter = pv.Plotter(notebook=True)
    plotter.set_background('grey')
    plotter.add_mesh(mesh, cmap='Blues', scalars=var)
    plotter.show(jupyter_backend='static')